## camera variance

In [12]:
import cv2
import numpy as np

dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
parameters =  cv2.aruco.DetectorParameters()

# Camera matrix and distortion coefficients (example values, replace with your own)
camera_matrix = np.array([[1000, 0, 640],
                          [0, 1000, 360],
                          [0, 0, 1]], dtype=np.float32)
dist_coeffs = np.zeros((5,))  # Assuming no distortion for simplicity; replace with actual values if available

# Marker size (in meters)
marker_length = 0.1  # Replace with your marker sizeq

# Arrays to store the translation and rotation data
translations = []
rotations = []

# Open the webcam
cap = cv2.VideoCapture(1)

print("Press 'q' to stop capturing frames and calculate variance.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Convert to grayscale for ArUco detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, dictionary, parameters=parameters)

    if ids is not None:
        # Estimate pose of the detected marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)

        # Draw marker and pose on the frame
        for rvec, tvec in zip(rvecs, tvecs):
            cv2.aruco.drawDetectedMarkers(frame, corners)
            #cv2.aruco.drawAxis(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.1)

            # Store the translation and rotation vectors
            translations.append(tvec[0])  # Extract the translation vector (x, y, z)
            rotations.append(rvec[0])    # Extract the rotation vector (Rodrigues)

    # Display the frame
    cv2.imshow('ArUco Detection', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

# Convert lists to numpy arrays for easier calculation
translations = np.array(translations)
rotations = np.array(rotations)

# Calculate variance for translation and rotation
translation_variance = np.var(translations, axis=0)
rotation_variance = np.var(rotations, axis=0)

# Display results
print("Translation Variance (x, y, z):", translation_variance)
print("Rotation Variance (Rodrigues x, y, z):", rotation_variance)

print("Translation Variance (x, y, z):", translation_variance*1e6)
print("Rotation Variance (Rodrigues x, y, z):", rotation_variance*(180/np.pi)**2)




Press 'q' to stop capturing frames and calculate variance.
Translation Variance (x, y, z): nan
Rotation Variance (Rodrigues x, y, z): nan
Translation Variance (x, y, z): nan
Rotation Variance (Rodrigues x, y, z): nan


In [34]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
Press 'q' to stop capturing frames and calculate variance.
Translation Variance (x, y, z): [3.39999552e-07 3.05553558e-08 2.49620193e-06]
Rotation Variance (Rodrigues x, y, z): [4.79935094e-05 7.04296345e-06 2.23639587e-04]
Translation Variance (x, y, z): [0.33999955 0.03055536 2.49620193]
Rotation Variance (Rodrigues x, y, z): [0.1575534  0.02312069 0.73416546]

## data acquisition

In [13]:
def get_data():
    thymio_data.append({"ground":list(node["prox.ground.reflected"]), 
                        "sensor":list(node["prox.ground.reflected"]),
                        "left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})
    


## prediction

In [29]:
def prediction_step(X,U,P,previous_t):

    current_t= time.time()
    if previous_t:
        dt=round(current_t-previous_t,4)
    else:
        time.sleep(0.2-dt)
    
    theta=X[2]
    A = np.eye(len(X))
    B = np.array([0.5*dt*np.cos(theta), 0.5*dt*np.cos(theta)],
                [0.5*dt*np.sin(theta), 0.5*dt*np.sin(theta)],
                [-dt/robot_length,dt/robot_length])
    x_new= A.dot(X) + B.dot(U)
    Q=np.eye(len(X))*speed_variance
    P += B.dot(Q).dot(B.T) 
    return X,P,current_t
    
    

## update 

In [33]:
def update_step(X,Z,P,aruco_detected):
    R=np.diag([translation_variance[0],translation_variance[1],rotation_variance[2]])
    R_nt=R*np.inf
    I=Z-X
    H = np.eye(len(X))
    if aruco_detected:
        S=H.dot(P).dot(H.T) + R
    else:
        S=H.dot(P).dot(H.T) + R_nt
    K_gain = P.dot(H.T).dot(np.linalg.inv(S))
    X += K_gain.dot(I)
    P -= K_gain.dot(H).dot(P)

    return X, P


## kalman main

In [30]:
import time
robot_length=100
speed_variance = 151.4387409522633
X=np.zeros(3)
U=np.zeros(2)
P=np.eye(3)*1
X=np.zeros(3)
Z = np.zeros(3)

NameError: name 'camera_variance' is not defined